In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import theano
# import theano.tensor as T


import os
import timeit
from datetime import datetime
from mpfntutils import load_data


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

In [2]:
def initalizeParameters(units = 32):
    """
    Initialize an initial symmetric W matrix from a random normal distribution with mean 0 and variance 1/n.
    Inputs:
    - n: size of the Boltzmann Machine (BM).
    """
    U = np.random.normal(loc = 0, scale = 1/units, size = (units, units))
    return 0.5 * (U + U.T)

In [3]:
# old cost and gradient    

# def Kcost(x, W, temperature = 1, units = 32):
#         """
#         Returns the cost computed by using the diagonals as the bias.
#         Inputs:
#         - x: samples used to train W.
#         - W: weights between the neurons of the Boltzmann Machine (BM).
#         - n: number of neurons in the BM.
#         - temperature: keep it as 1 until cost grows too big then raise temperature.
#         """
        
        
#         cost = np.mean(np.exp(1/temperature * ((0.5 - x) * np.dot(x, W)) \
#                               - 0.25 * np.diag(W)))
               
#         D = np.dot((0.5 - x).T, x)
#         Wgrad = cost * ((D + D.T) - (D * np.eye(units)) + 0.25 * np.eye(units))
#         return cost  
#         new one starts after here
        
def Kcost(x, W, temperature = 1):

    """
    Returns the cost computed by using the diagonals as the bias.
    Inputs:
    - x: samples used to train W.
    - W: weights between the neurons of the Boltzmann Machine (BM).
    - n: number of neurons in the BM.
    - temperature: keep it as 1 until cost grows too big then raise temperature.
    """
    num_samples = x.shape[0]        
    num_units = x.shape[1]
    delta = 1/2 - x
    diag = np.diag(W)[:, None].T
    E = delta * np.dot(x, W) - .25 * diag

    cost = np.sum(np.exp(1/temperature * E)) / num_samples         
    k = np.exp(E)        
    D = np.dot((delta * k).T, x)         
    C = np.zeros((num_units,))         
    np.copyto(C, np.diag(D))                 
    np.fill_diagonal(D, 0)         
    C = C - .25 * np.sum(k, axis = 0)         
    D = D + D.T         
    np.fill_diagonal(D, C) 

    return cost, D/ num_samples

In [ ]:
def params_adam_opt(grad, m = 0, v = 0, beta1 = 0.9, beta2 = 0.999):
    """
    Adaptive Moment Estimation (Adam) optimizer. Takes in parameters beta1, beta2 for adaptive 
    optimization. Returns next iteration of m and v for computing the grad update.
    
    Reference: http://arxiv.org/abs/1412.6980
    """
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)
    return m, v



In [4]:
def trainmpf(units = 32, lr = 1e-2, eps = 1e-8, n_epochs = 1000,
             batchsize = 32, temperature = 1, validate_every = 100, sample = '32-50K.npy'):
    """
    Trains parameters using MPF without using symbolic gradient.
    Inputs:
    - units: number of neurons in the BM.
    - lr: learning rate.
    - eps: parameter for Adam optimizer
    - n_epochs: number of epochs to train.
    - batchsize: size of batches for stochastic gradient descent.
    - temperature: happy to be 1 for now.
    - sample: sample used.
    
    """
    print (51 * '=')
    print (24 * '#' + 'MPF' + 24 * '#')
    print (51 * '=')
    print (str(datetime.now()))
    print ('Input size: {0}'.format(units))
    print ('Learning temperature: {0}'.format(temperature))
    print ('Learning rate: {0}'.format(lr))
    
    dataset = load_data(sample)
    n_dataset_batches = dataset.shape[0] // batchsize
    W = initalizeParameters(units = units)
    
    print ('Sample used: {0}'.format(sample))
    print ('=' * 51)
    
    start_time = timeit.default_timer()

# #     Extracting learnt parameters
#     W_learnt = np.zeros((units, units))
#     b_learnt = np.zeros((units,))
# #     Renaming to W_learnt and b_learnt
#     np.copyto(b_learnt, np.diag(W))
#     np.copyto(W_learnt, W)
#     np.fill_diagonal(W_learnt, 0)

#     Loading the original W and b
    org_W = np.load(sample[0:2] + '-' + 'W' + '.npy')
    org_b = np.load(sample[0:2] + '-' + 'b' + '.npy')
   
    
#     Tracking best parameters learnt
    best_mse = np.inf
    best_W = [None, np.inf]
    best_b = [None, np.inf]
    best_epoch = np.inf
    
    for epoch in range(n_epochs):
        c = []
        current_time = timeit.default_timer()
        for batch_index in range(n_dataset_batches):
            minibatch = dataset[batch_index * batchsize: (batch_index + 1) * batchsize]
            cost, grad = Kcost(dataset, W)
#             Extracting learnt parameters
            W_learnt = np.zeros((units, units))
            b_learnt = np.zeros((units,))
#             Renaming to W_learnt and b_learnt
            np.copyto(b_learnt, np.diag(W))
            np.copyto(W_learnt, W)
            np.fill_diagonal(W_learnt, 0)
            
#             m, v = params_adam_opt(grad)
#             W += -lr * m / (np.sqrt(v) + eps)
            W += -lr * grad
            c.append(cost)
            

        mseW = np.linalg.norm(org_W - W_learnt)/ ((units**2 - units) / 2)
        mseb = np.linalg.norm(org_b - b_learnt)/ units
        mse = mseW + mseb


        if mse < best_mse:
            best_mse = mse
            best_W[0] = W_learnt  
            best_W[1] = mseW
            best_b[0] = b_learnt  
            best_b[1] = mseb
            best_cost = np.mean(c, dtype='float64')
            best_epoch = epoch

        if epoch%validate_every == 0:
            print ('Training epoch %d/%d, Cost: %f mseW: %.5f, mseb: %.5f, mse: %.5f, Time Elasped: %.2f '\
                 % (epoch, n_epochs, np.mean(c, dtype='float64'), \
                 mseW, mseb, mse,  (current_time - start_time)/60) )
    
    end_time = timeit.default_timer()
    
    training_time = end_time - start_time
    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize=(20,10))
    fig.tight_layout()
    plt.setp(ax, xticks=np.arange(0,100,16))
    ax[0,0].plot(org_W.reshape(-1,1)[0:100], 'r')
#     ax[0,0].plot(W_learnt.reshape(-1,1)[0:100], 'b')
    ax[0,0].plot(best_W[0].reshape(-1,1)[0:100], 'g')
    ax[0,0].set_title('W')
    ax[0,0].legend(['W', 'Learnt W','Best W'])
    ax[0,1].plot(org_W.reshape(-1,1)[101:200], 'r')
#     ax[0,1].plot(W_learnt.reshape(-1,1)[101:200], 'b')
    ax[0,1].plot(best_W[0].reshape(-1,1)[101:200], 'g')
    ax[0,1].set_title('W')
    ax[0,1].legend(['W', 'Learnt W','Best W'])
    ax[1,0].plot(org_W.reshape(-1,1)[201:256], 'r')
#     ax[1,0].plot(W_learnt.reshape(-1,1)[201:256], 'b')
    ax[1,0].plot(best_W[0].reshape(-1,1)[201:256], 'g')
    ax[1,0].set_title('W')
    ax[1,0].legend(['W', 'Learnt W','Best W'])
    ax[1,1].plot(org_b.reshape(-1,1), 'r')
#     ax[1,1].plot(b_learnt.reshape(-1,1),'b')
    ax[1,1].plot(best_b[0].reshape(-1,1),'g')
    ax[1,1].set_title('b')
    ax[1,1].legend(['b', 'Learnt b','Best b'])

    
    print ('The training took %.2f minutes' % (training_time/60.))
    print ('#' * 22 + 'Results' + '#' * 22)
    print ('=' * 51)
    print ('Best mse: {0}'.format(best_mse))
    print ('Best W mse: {0}'.format(best_W[1]))
    print ('Best b mse: {0}'.format(best_b[1]))
    print ('Best epoch: {0}'.format(best_epoch))
    print ('=' * 51)


In [5]:
trainmpf(units = 16, lr = 1e-2, eps = 1e-8, n_epochs = 100,
             batchsize = 16, temperature = 1, validate_every = 10, sample = '16-50K.npy')

########################MPF########################
2017-06-01 08:53:16.608582
Input size: 16
Learning temperature: 1
Learning rate: 0.01
Sample used: 16-50K.npy
Training epoch 0/100, Cost: 3.871938 mseW: 0.07188, mseb: 1.01059, mse: 1.08247, Time Elasped: 0.00 
Training epoch 10/100, Cost: 0.203961 mseW: 0.08102, mseb: 2.09058, mse: 2.17161, Time Elasped: 21.70 
Training epoch 20/100, Cost: 0.106927 mseW: 0.08762, mseb: 2.40877, mse: 2.49639, Time Elasped: 46.73 
Training epoch 30/100, Cost: 0.072676 mseW: 0.09213, mseb: 2.60309, mse: 2.69521, Time Elasped: 69.78 


KeyboardInterrupt: 